# Load json velov

In [ ]:
# Open Json velov dataset 
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from datetime import datetime
import folium
from folium.plugins import HeatMap
import geopandas as gpd
from shapely.geometry import Point
import requests
import io
import os 
import sys 

sys.path.append(os.path.abspath(os.path.join(os.getcwd(),'..','..','..')))
from load_inputs.Lyon.pt.subway_in import get_trigram_correspondance

data_folder  = '../../../../../../data/rrochas/raw_data'
bike_json_path = 'VELO-LYON/locations/pvo_patrimoine_voirie.pvostationvelov.json'
subway_csv_path = 'keolis_data_2019-2020/ref_subway.csv'
# subway_lane_path = 'keolis_data_2019-2020/ref_lines.csv'
subway_lanes_path = 'keolis_data_2019-2020/metro_matching_TRI_FULL.CSV'




,TRI_STA,FULL_STA,LIGNE_A,LIGNE_B,LIGNE_C,LIGNE_D,max_line
0,AMP,AMPERE,2,0,0,0,LIGNE_A
1,BEL,BELLECOUR,3,0,0,5,LIGNE_D
2,BLA,GRANGE BLANCHE,0,0,0,11,LIGNE_D
3,BON,BONNEVAY,13,0,0,0,LIGNE_A
4,BRO,BROTTEAUX,0,2,0,0,LIGNE_B
...,...,...,...,...,...,...,...
59,BEL,BELLECOUR / A,3,0,0,0,LIGNE_A
60,BEL,BELLECOUR / D,0,0,0,5,LIGNE_D
61,SAX,SAXE GAMBETTA / B,0,5,0,0,LIGNE_B
62,SAX,SAXE GAMBETTA / D,0,0,0,7,LIGNE_D


In [ ]:
# Load Subway data
df_subway = pd.read_csv(os.path.join(data_folder, subway_csv_path),index_col =0)
subway_lanes = pd.read_csv(os.path.join(data_folder,subway_lanes_path),sep=';').rename(columns = {'LIGNE_A':'A', 'LIGNE_B':'B', 'LIGNE_C':'C', 'LIGNE_D':'D'})
subway_lanes['max_line'] = subway_lanes[['A','B','C','D']].idxmax(1)
laneA = list(set(subway_lanes[subway_lanes['max_line']=='A']['TRI_STA'].tolist()))
laneB = list(set(subway_lanes[subway_lanes['max_line']=='B']['TRI_STA'].tolist()))
laneC = list(set(subway_lanes[subway_lanes['max_line']=='C']['TRI_STA'].tolist()))
laneD = list(set(subway_lanes[subway_lanes['max_line']=='D']['TRI_STA'].tolist()))

df_subway_A = df_subway[df_subway['COD_TRG'].isin(laneA)]
df_subway_B = df_subway[df_subway['COD_TRG'].isin(laneB)]
df_subway_C = df_subway[df_subway['COD_TRG'].isin(laneC)]
df_subway_D = df_subway[df_subway['COD_TRG'].isin(laneD)]

colorA = 'red'
colorB = 'blue'
colorC = 'green'
colorD = 'yellow'

df_correspondance = get_trigram_correspondance()

In [73]:
# Load Bike data
with open(os.path.join(data_folder, bike_json_path)) as f:
    bike_data = json.load(f)
df_bike = pd.json_normalize(bike_data['features'])
df_bike['geometry.coordinates'] = df_bike['geometry.coordinates'].apply(lambda x: (x[1],x[0]))
df_bike['station_id'] = df_bike['properties.idstation']
df_bike['geometry'] = df_bike['geometry.coordinates'].apply(lambda x: Point(x[1],x[0]))
df_bike = df_bike.set_geometry('geometry')
df_bike = df_bike.set_crs(epsg=4326)
df_bike = df_bike[['station_id','geometry']]
df_bike.explore()

,IDT_SIT,MEAN_X,MEAN_Y,COD_TRG,COD_STA,LIB_STA_SIFO
0,5000.0,4.829212,45.753114,AMP,AMP,AMPERE
1,5002.0,4.833397,45.757263,BEL,BEL,BELLECOUR
3,5021.0,4.835813,45.763449,COR,COR,CORDELIERS
5,5023.0,4.900288,45.765709,CUS,CUS,CUSSET
6,5024.0,4.890049,45.767590,FLA,FLA,FLACHET
9,5042.0,4.882185,45.769037,GRA,GRA,GRATTE CIEL
13,5046.0,4.836024,45.767484,HOT,HOT,HOTEL DE VILLE
16,5081.0,4.853372,45.769584,MAS,MAS,MASSENA
20,5085.0,4.826951,45.749782,PER,PER,PERRACHE
25,5095.0,4.826484,45.760031,JEA,JEA,VIEUX LYON
